In [1]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# STAGE 1. Data Preprocessing

- 참고: https://github.com/feng-123/Enhancer-LSTMAtt

## 검증 가설

1. 주어진 DNA sequence의 Enhancer sequence 여부를 NLP 신경망 모델을 통해 판별할 수 있다.
2. 주어진 Enhancer sequence를 통해 Strong enhancer인지 Weak enhancer인지 NLP 신경망 모델을 통해 판별할 수 있다.

## Enhancer Sequence Preprocessing

Train Dataset
- enhancer.cv.txt: Enhancer Sequence 1484개
  - strong_742.txt: Strong Enhancer Sequence 742개
  - weak_742.txt  : Weak Enhancer Sequence 742개
- non.cv.txt     : Non-Enhancer Sequence 1484개

Test Dataset
- enhancer.ind.txt: Enhancer Sequence 200개
  - strong_100.txt : Strong Enhancer Sequence 100개
  - weak_100.txt   : Weak Enhancer Sequence 100개
- non.ind.txt     : Non-Enhancer Sequence 200개

In [2]:
import os
import numpy as np

In [3]:
### 파일 경로 설정

train_dir = "drive/MyDrive/aib-section4-project/dataset/train/"
test_dir = "drive/MyDrive/aib-section4-project/dataset/test/"

In [4]:
### 전부 함수로 정의 후 한꺼번에 전처리
### 참고 코드: https://github.com/feng-123/Enhancer-LSTMAtt

def readSequence():
    
    ## Enhancer & Non-Enhancer Sets
    with open(os.path.join(train_dir, "enhancer.cv.txt")) as f:
        enhancer_cv = f.readlines()
        enhancer_cv = [s.strip() for s in enhancer_cv]
    with open(os.path.join(train_dir, "non.cv.txt")) as f:
        non_cv = f.readlines()
        non_cv = [s.strip() for s in non_cv]
    with open(os.path.join(test_dir, "enhancer.ind.txt")) as f:
        enhancer_ind = f.readlines()
        enhancer_ind = [s.strip() for s in enhancer_ind]
    with open(os.path.join(test_dir, "non.ind.txt")) as f:
        non_ind = f.readlines()
        non_ind = [s.strip() for s in non_ind]
    
    ## Strong/Weak Enhancer Sets
    with open(os.path.join(train_dir, "strong_742.txt")) as f:
        strong_742 = f.readlines()
        strong_742 = [s.strip() for s in strong_742]
    with open(os.path.join(train_dir, "weak_742.txt")) as f:
        weak_742 = f.readlines()
        weak_742 = [s.strip() for s in weak_742]
    with open(os.path.join(test_dir, "strong_100.txt")) as f:
        strong_100 = f.readlines()
        strong_100 = [s.strip() for s in strong_100]
    with open(os.path.join(test_dir, "weak_100.txt")) as f:
        weak_100 = f.readlines()
        weak_100 = [s.strip() for s in weak_100]

    return enhancer_cv, non_cv, enhancer_ind, non_ind, strong_742, weak_742, strong_100, weak_100


def removeName_PN(data): ## Enhancer(Positive) / Non-Enhancer(Negative) Sequence의 이름 제거
    data_new = []
    for i in range(1,len(data),2):
        data_new.append(data[i])
    return data_new

def removeName_SW(data): ## Strong/Weak Enhancer Sequence의 이름 제거
    data_new = []
    for i in range(1,len(data),5):
        data_new.append(data[i].upper()+data[i+1].upper()+data[i+2].upper()+data[i+3].upper())
    return data_new


def GetSets():
    enhancer_cv, non_cv, enhancer_ind, non_ind, strong_742, weak_742, strong_100, weak_100 = readSequence()

    enhancer_cv = removeName_PN(enhancer_cv)
    non_cv = removeName_PN(non_cv)
    enhancer_ind = removeName_PN(enhancer_ind)
    non_ind = removeName_PN(non_ind)
    X_train_pn = np.concatenate([enhancer_cv, non_cv], axis=0)
    X_test_pn = np.concatenate([enhancer_ind, non_ind], axis=0)
    y_train_pn = np.concatenate([np.ones((len(enhancer_cv),)), np.zeros((len(non_cv),))], axis=0)
    y_test_pn = np.concatenate([np.ones((len(enhancer_ind),)), np.zeros((len(non_ind),))], axis=0)
    
    strong_742 = removeName_SW(strong_742)
    weak_742 = removeName_SW(weak_742)
    strong_100 = removeName_SW(strong_100)
    weak_100 = removeName_SW(weak_100)
    X_train_sw = np.concatenate([strong_742, weak_742], axis=0)
    X_test_sw = np.concatenate([strong_100, weak_100], axis=0)
    y_train_sw = np.concatenate([np.ones((len(strong_742),)), np.zeros((len(weak_742),))], axis=0)
    y_test_sw = np.concatenate([np.ones((len(strong_100),)), np.zeros((len(weak_100),))], axis=0)

    return X_train_pn, X_test_pn, y_train_pn, y_test_pn, X_train_sw, X_test_sw, y_train_sw, y_test_sw

In [5]:
X_train_pn, X_test_pn, y_train_pn, y_test_pn, X_train_sw, X_test_sw, y_train_sw, y_test_sw = GetSets()

## Importing the Pre-Trained Word2Vec Model: "dna2vec"

Enhancer Sequence를 벡터화해 주는 사전학습 Word2Vec 모델 불러오기
- 출처: https://github.com/pnpnpn/dna2vec
<br>
<br>

***모델 소스코드(6년 전 코드)에서 텐서플로우 구버전 코드 수정***
- 코드 수정 레퍼런스
  - https://stackoverflow.com/questions/42363897/attributeerror-type-object-word2vec-has-no-attribute-load-word2vec-format
  - https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4
  - https://stackoverflow.com/questions/66868221/gensim-3-8-0-to-gensim-4-0-0
  - https://radimrehurek.com/gensim/models/word2vec.html

In [6]:
!pip install logbook

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!pip install gensim --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
from __future__ import print_function
import logbook
import tempfile
import numpy as np

# from gensim.models import word2vec
from gensim.models import KeyedVectors
from gensim import matutils

class SingleKModel:
    def __init__(self, model):
        self.model = model
        self.vocab_lst = sorted(model.key_to_index.keys())
        ###원 코드: self.vocab_lst = sorted(model.vocab.keys())

class MultiKModel:
    def __init__(self, filepath):
        self.aggregate = KeyedVectors.load_word2vec_format(filepath, binary=False)
        ###원 코드: self.aggregate = word2vec.Word2Vec.load_word2vec_format(filepath, binary=False)
        self.logger = logbook.Logger(self.__class__.__name__)

        vocab_lens = [len(vocab) for vocab in self.aggregate.key_to_index.keys()]
        ###원 코드: vocab_lens = [len(vocab) for vocab in self.aggregate.vocab.keys()]
        self.k_low = min(vocab_lens)
        self.k_high = max(vocab_lens)
        self.vec_dim = self.aggregate.vector_size

        self.data = {}
        for k in range(self.k_low, self.k_high + 1):
            self.data[k] = self.separate_out_model(k)

    def model(self, k_len):
        """
        Use vector('ACGTA') when possible
        """
        return self.data[k_len].model

    def vector(self, vocab):
        return self.data[len(vocab)].model[vocab]

    def unitvec(self, vec):
        return matutils.unitvec(vec)

    def cosine_distance(self, vocab1, vocab2):
        return np.dot(self.unitvec(self.vector(vocab1)), self.unitvec(self.vector(vocab2)))

    def l2_norm(self, vocab):
        return np.linalg.norm(self.vector(vocab))

    def separate_out_model(self, k_len):
        vocabs = [vocab for vocab in self.aggregate.key_to_index.keys() if len(vocab) == k_len]
        ###원 코드: vocabs = [vocab for vocab in self.aggregate.vocab.keys() if len(vocab) == k_len]
        if len(vocabs) != 4 ** k_len:
            self.logger.warn('Missing {}-mers: {} / {}'.format(k_len, len(vocabs), 4 ** k_len))

        header_str = '{} {}'.format(len(vocabs), self.vec_dim)
        with tempfile.NamedTemporaryFile(mode='w') as fptr:
            print(header_str, file=fptr)
            for vocab in vocabs:
                vec_str = ' '.join("%f" % val for val in self.aggregate[vocab])
                print('{} {}'.format(vocab, vec_str), file=fptr)
            fptr.flush()
            return SingleKModel(KeyedVectors.load_word2vec_format(fptr.name, binary=False))
            ###원 코드: return SingleKModel(word2vec.Word2Vec.load_word2vec_format(fptr.name, binary=False))

In [9]:
### 모델 불러오기
filename = 'dna2vec-20161219-0153-k3to8-100d-10c-29320Mbp-sliding-Xat.w2v'
filepath = os.path.join('drive/MyDrive/aib-section4-project/pretrained', filename)

### Model Instanciation
d2v = MultiKModel(filepath)

## Tokenizing & Embedding

In [10]:
### 모두 함수로 정의

def tokenizer(sequence): ## 3-mer로 Tokenize
    token_list = []
    for i in range(len(sequence)-2): ## 마지막 3-mer까지 얻으려면 길이에서 2뺀 걸 마지막 starting index로 해야 함!
        token = sequence[i:i+3]
        token_list.append(token)

    return token_list


def tokenizeSeqs(dataset): ## 데이터셋을 Tokenize
    tokenset_list = []
    for sequence in dataset:
        token_list = tokenizer(sequence)
        tokenset_list.append(token_list)

    tokenset_array = np.array(tokenset_list) ### array화 - n131 참조
    return tokenset_array


def embedding(dataset): ## 데이터셋의 각 Token을 Embedding vectors로
    data_list = []
    for tokenset in dataset:
        embedding_list = []
        for token in tokenset:
            vector = d2v.vector(token)
            embedding_list.append(vector)
        data_list.append(embedding_list)

    dataset_array = np.array(data_list)
    return dataset_array

In [11]:
X_train_tokenized_pn = tokenizeSeqs(X_train_pn)
X_train_embedded_pn = embedding(X_train_tokenized_pn)

X_test_tokenized_pn = tokenizeSeqs(X_test_pn)
X_test_embedded_pn = embedding(X_test_tokenized_pn)

X_train_tokenized_sw = tokenizeSeqs(X_train_sw)
X_train_embedded_sw = embedding(X_train_tokenized_sw)

X_test_tokenized_sw = tokenizeSeqs(X_test_sw)
X_test_embedded_sw = embedding(X_test_tokenized_sw)

# STAGE 2. Baseline Model (Chance Level)

가설 2개 모두 훈련세트, 테스트세트 각각의 타겟 클래스 비율이 0.5로 동일하게 맞춰져 있음.
- 따라서 **Chance Level = 0.5**

<br>
<br>

# STAGE 3. Loading the Model Architecture of iEnhancer-CNN

모델 'iEnhancer-CNN' 논문의 모델 아키텍처를 가져옴.
- 논문 링크(open access): https://ieeexplore.ieee.org/abstract/document/9044822
- 소스코드는 따로 없으므로 모두 직접 작성

***논문에 Flatten layer를 썼다는 이야기는 없으나 output shape를 보면 Dense layer 직전에 필수적인 것으로 보여 추가함.***


In [12]:
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv1D, Dropout, Flatten, Dense, Activation

In [13]:
### 아래에서 GridSearchCV를 사용하기 위해 모델을 함수로 정의

def create_model():
  input = Input(shape=(198,100))
  out = Conv1D(8, 7, strides=1, padding='same')(input)
  out = Activation('relu')(out)
  out = Conv1D(8, 7, strides=1, padding='same')(out)
  out = Activation('relu')(out)
  out = Dropout(0.5)(out)
  out = Flatten()(out)
  output = Dense(1, activation='sigmoid')(out)
  
  model = Model(inputs=input, outputs=output)
  model.compile(
      loss='binary_crossentropy', 
      optimizer=tf.keras.optimizers.Adam(learning_rate=8*1e-4), 
      metrics=['accuracy']
  )

  return model

In [14]:
model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 198, 100)]        0         
                                                                 
 conv1d (Conv1D)             (None, 198, 8)            5608      
                                                                 
 activation (Activation)     (None, 198, 8)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 198, 8)            456       
                                                                 
 activation_1 (Activation)   (None, 198, 8)            0         
                                                                 
 dropout (Dropout)           (None, 198, 8)            0         
                                                                 
 flatten (Flatten)           (None, 1584)              0     

In [15]:
### 학습 진행해 보기 - 가설1(P/N)
### batch+size, epochs는 논문과 같게 설정

model.fit(X_train_embedded_pn, y_train_pn, batch_size=32, epochs=50, verbose=1)

Epoch 1/50
93/93 [==============================] - 4s 7ms/step - loss: 0.6553 - accuracy: 0.5910
Epoch 2/50
93/93 [==============================] - 0s 5ms/step - loss: 0.5360 - accuracy: 0.7274
Epoch 3/50
93/93 [==============================] - 0s 5ms/step - loss: 0.5148 - accuracy: 0.7500
Epoch 4/50
93/93 [==============================] - 1s 6ms/step - loss: 0.5129 - accuracy: 0.7419
Epoch 5/50
93/93 [==============================] - 1s 7ms/step - loss: 0.5026 - accuracy: 0.7584
Epoch 6/50
93/93 [==============================] - 1s 8ms/step - loss: 0.4923 - accuracy: 0.7648
Epoch 7/50
93/93 [==============================] - 1s 8ms/step - loss: 0.4844 - accuracy: 0.7679
Epoch 8/50
93/93 [==============================] - 1s 8ms/step - loss: 0.4748 - accuracy: 0.7746
Epoch 9/50
93/93 [==============================] - 1s 9ms/step - loss: 0.4595 - accuracy: 0.7830
Epoch 10/50
93/93 [==============================] - 1s 8ms/step - loss: 0.4510 - accuracy: 0.7891
Epoch 11/50
93/93 [

In [16]:
model.evaluate(X_test_embedded_pn, y_test_pn)

13/13 [==============================] - 0s 12ms/step - loss: 0.6341 - accuracy: 0.7625


[0.634096086025238, 0.762499988079071]

In [17]:
### 학습 진행 - 가설2(S/W)

model = create_model()
model.fit(X_train_embedded_sw, y_train_sw, batch_size=32, epochs=50, verbose=1)

Epoch 1/50
47/47 [==============================] - 1s 8ms/step - loss: 0.6979 - accuracy: 0.5047
Epoch 2/50
47/47 [==============================] - 0s 5ms/step - loss: 0.6879 - accuracy: 0.5283
Epoch 3/50
47/47 [==============================] - 0s 5ms/step - loss: 0.6776 - accuracy: 0.5761
Epoch 4/50
47/47 [==============================] - 0s 4ms/step - loss: 0.6594 - accuracy: 0.6051
Epoch 5/50
47/47 [==============================] - 0s 4ms/step - loss: 0.6312 - accuracy: 0.6415
Epoch 6/50
47/47 [==============================] - 0s 5ms/step - loss: 0.6150 - accuracy: 0.6637
Epoch 7/50
47/47 [==============================] - 0s 5ms/step - loss: 0.5936 - accuracy: 0.6813
Epoch 8/50
47/47 [==============================] - 0s 5ms/step - loss: 0.5702 - accuracy: 0.7055
Epoch 9/50
47/47 [==============================] - 0s 4ms/step - loss: 0.5553 - accuracy: 0.7230
Epoch 10/50
47/47 [==============================] - 0s 4ms/step - loss: 0.5357 - accuracy: 0.7352
Epoch 11/50
47/47 [

In [18]:
model.evaluate(X_test_embedded_sw, y_test_sw)

7/7 [==============================] - 0s 18ms/step - loss: 0.5218 - accuracy: 0.7600


[0.5218212008476257, 0.7599999904632568]

# STAGE 4. Fitting the Model by Cross-Validation

## Evaluation Metrics

실제 학습에서는 Accuracy 외에 논문에서 지정한 다른 Evaluation 파라미터를 사용.
- 이미지 링크: https://drive.google.com/file/d/12-fKOkVI2NeYJxbkMr5sFn_PFDj2Hfe5/view
- 출처 논문 링크(open access): https://academic.oup.com/bioinformatics/article/32/3/362/1744331

In [19]:
### 참고 코드: https://github.com/feng-123/Enhancer-LSTMAtt

import math

def evaluation_metrics(y_true, y_pred): 

    pos_num = np.sum(y_true==1)
    print('pos_num=',pos_num)

    neg_num = y_true.shape[0] - pos_num
    print('neg_num=',neg_num)

    tp =np.sum((y_true==1) & (y_pred==1))
    print('tp=',tp)

    tn = np.sum(y_true==y_pred) - tp
    print('tn=',tn)

    sn = tp / pos_num
    sp = tn / neg_num

    acc = (tp+tn) / (pos_num + neg_num)

    fn = pos_num - tp
    fp = neg_num - tn
    print('fn=',fn)
    print('fp=',fp)
    
    tp = np.array(tp)
    tn = np.array(tn)
    fp = np.array(fp)
    fn = np.array(fn)
    mcc = (tp*tn - fp*fn) / (np.sqrt((tp+fn)*(tp+fp)*(tn+fp)*(tn+fn)))

    return sn, sp, acc, mcc

## K-Fold Cross-Validation w/ GridSearch

논문에서 Layer의 파라미터는 다 최적화한 상태이므로 batch size만 튜닝하면서 K-Fold CV를 함께 진행.
- 논문처럼 k=5로 설정.

In [20]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [21]:
model = KerasClassifier(build_fn=create_model, verbose=1)

batch_size = [8, 16, 32, 64]
epochs = [50]
param_grid = dict(batch_size=batch_size, epochs=epochs)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


### Hypothesis 1

In [22]:
### 가설1(P/N)

cv = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, verbose=1, n_jobs=-1)
cv_pn_result = cv.fit(X_train_embedded_pn, y_train_pn)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Epoch 1/50
371/371 [==============================] - 2s 4ms/step - loss: 0.5915 - accuracy: 0.6691
Epoch 2/50
371/371 [==============================] - 1s 3ms/step - loss: 0.5215 - accuracy: 0.7476
Epoch 3/50
371/371 [==============================] - 1s 3ms/step - loss: 0.5061 - accuracy: 0.7554
Epoch 4/50
371/371 [==============================] - 1s 3ms/step - loss: 0.4868 - accuracy: 0.7685
Epoch 5/50
371/371 [==============================] - 1s 3ms/step - loss: 0.4727 - accuracy: 0.7793
Epoch 6/50
371/371 [==============================] - 1s 3ms/step - loss: 0.4527 - accuracy: 0.7867
Epoch 7/50
371/371 [==============================] - 1s 3ms/step - loss: 0.4388 - accuracy: 0.8076
Epoch 8/50
371/371 [==============================] - 1s 3ms/step - loss: 0.4232 - accuracy: 0.8083
Epoch 9/50
371/371 [==============================] - 1s 3ms/step - loss: 0.3931 - accuracy: 0.8221
Epoch 10/50
371/371 [==============================] - 1s 3ms/step - loss: 0.3931 - accuracy: 0.8221

In [23]:
print(f"Best: {cv_pn_result.best_score_} using {cv_pn_result.best_params_}")

Best: 0.6320949733257294 using {'batch_size': 8, 'epochs': 50}


### Hypothesis 2

In [24]:
### 가설2(S/W)

cv2 = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1)
cv_sw_result = cv2.fit(X_train_embedded_sw, y_train_sw)

Epoch 1/50
186/186 [==============================] - 1s 4ms/step - loss: 0.6963 - accuracy: 0.5020
Epoch 2/50
186/186 [==============================] - 1s 4ms/step - loss: 0.6756 - accuracy: 0.5869
Epoch 3/50
186/186 [==============================] - 1s 4ms/step - loss: 0.6400 - accuracy: 0.6368
Epoch 4/50
186/186 [==============================] - 1s 4ms/step - loss: 0.5998 - accuracy: 0.6806
Epoch 5/50
186/186 [==============================] - 1s 4ms/step - loss: 0.5756 - accuracy: 0.7008
Epoch 6/50
186/186 [==============================] - 1s 4ms/step - loss: 0.5397 - accuracy: 0.7352
Epoch 7/50
186/186 [==============================] - 1s 4ms/step - loss: 0.4980 - accuracy: 0.7581
Epoch 8/50
186/186 [==============================] - 1s 4ms/step - loss: 0.4862 - accuracy: 0.7547
Epoch 9/50
186/186 [==============================] - 1s 4ms/step - loss: 0.4542 - accuracy: 0.7864
Epoch 10/50
186/186 [==============================] - 1s 4ms/step - loss: 0.4250 - accuracy: 0.8005

In [25]:
print(f"Best: {cv_sw_result.best_score_} using {cv_sw_result.best_params_}")

Best: 0.44129583835601804 using {'batch_size': 8, 'epochs': 50}


In [26]:
model_pn = cv.best_estimator_
model_sw = cv2.best_estimator_

# STAGE 5. Testing the Model

## Visualization

In [27]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

In [28]:
### ROC Curve로 결과를 시각화하는 함수 정의

def plot_roc_curve(y_true, y_pred):
  fpr, tpr, thresholds = roc_curve(y_true, y_pred)
  auc = auc(fpr, tpr)
  
  plt.figure(figsize=(5,5))
  plt.title('ROC curves')
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.xlim([-0.05,1.05])
  plt.ylim([-0.05,1.05])
  plt.plot(fpr, tpr, color='r')
  plt.plot([0, 1], [0, 1], color='m', linestyle='--')
  plt.show()

## Hypothesis 1. Enhancer vs. Non-Enhancer

In [29]:
y_pred_pn = model_pn.predict(X_test_embedded_pn)
### predict array shape이 (400, 1)로 레이블마다 []가 들어가므로 reshape

y_pred_pn = y_pred_pn.reshape((400,))

In [30]:
sn1, sp1, acc1, mcc1 = evaluation_metrics(y_test_pn, y_pred_pn)
print(f"sn={sn1}")
print(f"sp={sp1}")
print(f"acc={acc1}")
print(f"mcc={mcc1}")

pos_num= 200
neg_num= 200
tp= 167
tn= 140
fn= 33
fp= 60
sn=0.835
sp=0.7
acc=0.7675
mcc=0.539942853687922


In [31]:
# plot_roc_curve(y_test_pn, y_pred_pn)

### 시각화하려 했으나 RecursionError가 남.
### https://stackoverflow.com/questions/3323001/what-is-the-maximum-recursion-depth-in-python-and-how-to-increase-it

### 위 페이지 참고해서 recursion limit을 조금씩 올렸지만 계속해서 에러 나다가 100,000으로 설정 후 RAM 용량 초과했는지 Crash 발생 후 Runtime 재시작됨... 중단.

## Hypothesis 2. Strong vs. Weak Enhancer

In [32]:
y_pred_sw = model_sw.predict(X_test_embedded_sw)
y_pred_sw = y_pred_sw.reshape((200,))

In [33]:
sn2, sp2, acc2, mcc2 = evaluation_metrics(y_test_sw, y_pred_sw)
print(f"sn={sn2}")
print(f"sp={sp2}")
print(f"acc={acc2}")
print(f"mcc={mcc2}")

pos_num= 100
neg_num= 100
tp= 100
tn= 60
fn= 0
fp= 40
sn=1.0
sp=0.6
acc=0.8
mcc=0.6546536707079772


# STAGE 6. Alternative Model: Enhancer-LSTMAtt (Pre-Trained)

## Model Architecture

소스코드가 공개되어 있음. 변수명과 오류 나는 부분만 수정.
- 논문 링크(open access): https://www.mdpi.com/2218-273X/12/7/995
- 소스코드: https://github.com/feng-123/Enhancer-LSTMAtt

In [34]:
### 이 모델에서는 Word2Vec 모델을 쓰지 않고 다른 방식으로 인코딩 후 모델에서 Embedding layer를 사용함.

def encode_matrix(seq_matrix):
  ind_to_char = ['A','T','C','G','N']
  char_to_ind = {char: i for i, char in enumerate(ind_to_char)}
  
  return [[char_to_ind[i] for i in s] for s in seq_matrix]

In [35]:
X_train_encoded_pn = encode_matrix(X_train_pn)
X_test_encoded_pn = encode_matrix(X_test_pn)
X_train_encoded_sw = encode_matrix(X_train_sw)
X_test_encoded_sw = encode_matrix(X_test_sw)

X_train_encoded_pn = np.array(X_train_encoded_pn)
X_test_encoded_pn = np.array(X_test_encoded_pn)
X_train_encoded_sw = np.array(X_train_encoded_sw)
X_test_encoded_sw = np.array(X_test_encoded_sw)

In [36]:
from tensorflow.keras import backend as K
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Embedding,Dense,Flatten,Dropout,Add,Bidirectional,LSTM,Conv1D,GlobalMaxPool1D,MaxPooling1D,BatchNormalization,Activation,Reshape

In [37]:
class Attention3d(Layer):

    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2
            hidden = LSTM(64, return_sequences=True)(words)
            sentence = Attention()(hidden)
            # next add a Dense layer (for classification/regression) or whatever...
        """
        #self.supports_masking = True
        

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0

        super(Attention3d, self).__init__(**kwargs)

    def get_config(self):
         config = {"W_regularizer":self.W_regularizer,
                   "b_regularizer":self.b_regularizer,"W_constraint":self.W_constraint,"b_constraint":self.b_constraint,
                    "bias":self.bias,"step_dim":self.step_dim,"features_dim":self.features_dim}
         base_config = super(Attention3d, self).get_config()
         return dict(list(base_config.items()) + list(config.items()))

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1],),
                                 initializer=initializers.get('glorot_uniform'),
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        e = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))  # e = K.dot(x, self.W)
        if self.bias:
            e += self.b
        e = K.tanh(e)

        a = K.exp(e)
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())
        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)

        c = K.sum(a * x, axis=1)
        return c

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.features_dim

In [38]:
def resnet_identity_block(input_data, filters, kernel_size):
    x = Conv1D(filters, kernel_size, strides=1, padding='same')(input_data)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(filters, kernel_size, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = Add()([x, input_data])
    x = Activation('relu')(x)
    return x

def resnet_convolutional_block(input_data, filters, kernel_size):
    x = Conv1D(filters, kernel_size, strides=2, padding='valid')(input_data)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    X = Conv1D(filters, kernel_size, strides=2, padding='valid')(input_data)
    x = Add()([x, X])
    x = Activation('relu')(x)
    return x

In [62]:
def define_model():
    maxlen = 200
    max_features = 5
    embedding_dims = 32
    class_num = 1
    last_activation = 'sigmoid'
    input = Input((maxlen,))
    embedding = Embedding(max_features, embedding_dims, input_length=maxlen)(input)
    y = Conv1D(32, 8, strides=1, padding='same')(embedding)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPooling1D(pool_size=2, strides=1)(y)
    y = resnet_convolutional_block(y, 64, 8)
    y = resnet_identity_block(y, 64, 8)
    y = resnet_identity_block(y, 64, 8) 
    y = GlobalMaxPool1D()(y)

    x = Bidirectional(LSTM(32, return_sequences=True))(embedding)  # LSTM
    x = Bidirectional(LSTM(32, return_sequences=True))(x)
    x = Attention3d(maxlen)(x)
    x = Dropout(0.5)(x)

    t = tf.keras.layers.Concatenate()([x,y])
    t = Dense(16,activation='relu')(t)
    output = Dense(class_num, activation=last_activation)(t)
    model = Model(inputs=input, outputs=output)

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  metrics=['accuracy'])
    ###원 코드에서 loss=tf.keras.losses.BinaryCrossentropy(from_logits=True)
    ###--> UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
    ###수정함

    return model

## Testing Hypotheses

교차검증 및 하이퍼파라미터 튜닝은 생략.

*batch_size, epochs는 논문에 따로 명시가 안 되어 있음.*
- 위의 모델링에서 best parameter로 나온 batch_size=8 사용.
- epochs는 모델 성능 향상 정도를 보고 과적합 안 될 정도로 조정해서 학습.

### Hypothesis 1

In [60]:
model_pn = define_model()
model_pn.summary()

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 200)]        0           []                               
                                                                                                  
 embedding_9 (Embedding)        (None, 200, 32)      160         ['input_14[0][0]']               
                                                                                                  
 conv1d_80 (Conv1D)             (None, 200, 32)      8224        ['embedding_9[0][0]']            
                                                                                                  
 batch_normalization_63 (BatchN  (None, 200, 32)     128         ['conv1d_80[0][0]']              
 ormalization)                                                                             

In [84]:
model_pn = define_model()
model_pn.fit(X_train_encoded_pn, y_train_pn, batch_size=8, epochs=5, verbose=1)

Epoch 1/5
371/371 [==============================] - 17s 30ms/step - loss: 0.6604 - accuracy: 0.6792
Epoch 2/5
371/371 [==============================] - 11s 30ms/step - loss: 0.4447 - accuracy: 0.7982
Epoch 3/5
371/371 [==============================] - 11s 30ms/step - loss: 0.3162 - accuracy: 0.8898
Epoch 4/5
371/371 [==============================] - 11s 29ms/step - loss: 0.2097 - accuracy: 0.9515
Epoch 5/5
371/371 [==============================] - 11s 30ms/step - loss: 0.1317 - accuracy: 0.9845


In [85]:
### 소스코드 가져옴 (*y_pred 값이 확률값으로 나와서 조정을 해주는 코드)
### 임계값 수치가 0.5, 0.6으로 잘못 나와 있어서 수정

### 근데 왜 활성화함수가 시그모이드인데 확률값으로 나오지...?

res1 = model_pn.predict(X_test_encoded_pn)
pred1 = np.squeeze(res1, axis=-1)
f1 = pred1>0.5
pred1[f1] = 1
pred1[pred1<=0.5] = 0

y_pred_pn = pred1

In [86]:
sn1, sp1, acc1, mcc1 = evaluation_metrics(y_test_pn, y_pred_pn)
print(f"sn={sn1}")
print(f"sp={sp1}")
print(f"acc={acc1}")
print(f"mcc={mcc1}")

pos_num= 200
neg_num= 200
tp= 155
tn= 162
fn= 45
fp= 38
sn=0.775
sp=0.81
acc=0.7925
mcc=0.5853586420360279


### Hypothesis 2

In [69]:
model_sw = define_model()
model_sw.fit(X_train_encoded_sw, y_train_sw, batch_size=8, epochs=5, verbose=1)

Epoch 1/5
186/186 [==============================] - 14s 35ms/step - loss: 0.7786 - accuracy: 0.5499
Epoch 2/5
186/186 [==============================] - 10s 53ms/step - loss: 0.4740 - accuracy: 0.8275
Epoch 3/5
186/186 [==============================] - 8s 41ms/step - loss: 0.3197 - accuracy: 0.9589
Epoch 4/5
186/186 [==============================] - 7s 39ms/step - loss: 0.2225 - accuracy: 0.9892
Epoch 5/5
186/186 [==============================] - 6s 33ms/step - loss: 0.1594 - accuracy: 0.9980


In [78]:
res2 = model_pn.predict(X_test_encoded_sw)
pred2 = np.squeeze(res2, axis=-1)
f2 = pred2>0.5
pred2[f2] = 1
pred2[pred2<=0.5] = 0

y_pred_sw = pred2

In [79]:
sn2, sp2, acc2, mcc2 = evaluation_metrics(y_test_sw, y_pred_sw)
print(f"sn={sn2}")
print(f"sp={sp2}")
print(f"acc={acc2}")
print(f"mcc={mcc2}")

pos_num= 100
neg_num= 100
tp= 100
tn= 37
fn= 0
fp= 63
sn=1.0
sp=0.37
acc=0.685
mcc=0.47643873166512696
